In [1]:
import os 
os.chdir('..')

%load_ext autoreload
%autoreload 2

from src.babylm_baseline_train.datasets.babyLM import get_babyLM_10M, BabyLM
import matplotlib.pyplot as plt
import torch
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import math
from collections import defaultdict
import numpy as np

/home/misra/anaconda3/envs/babylm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset(path=os.path.join('./src/babylm_baseline_train/datasets', "babyLM_for_hf.py"),
            name='babyLM-10M',
            split='train')

Found cached dataset baby_lm_for_hf (/home/misra/.cache/huggingface/datasets/baby_lm_for_hf/babyLM-10M/1.0.0/281c1a7c3ebf0b682e9bdca60f4a2442b6aaf2d2a266fea843461e98f10a5f07)


In [3]:
tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizers/babylm_10M_BPE.json")

In [4]:
tokenizer.add_special_tokens({
    'bos_token': '<s>', 
    'eos_token': '</s>',
    'unk_token': '<unk>',
    'pad_token': '<pad>',
    'mask_token' : '<mask>'  
})

0

In [5]:
def preprocess_function(example):
    encoded = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    
    return {
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"],
#         "token_type_ids": encoded["token_type_ids"],
    }


In [6]:
dataset = dataset.map(preprocess_function, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

In [14]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [15]:
next(iter(dataloader))

{'input_ids': tensor([[ 554,  316,  265, 1136,   35,  203,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      

In [22]:
from collections import defaultdict
import math

def calculate_tf(corpus):
    tf_counts = defaultdict(int)
    total_documents = len(corpus)

    for instance in tqdm(corpus):
        tokens = instance['input_ids'].squeeze().tolist()
        for token in tokens:
            tf_counts[token] += 1

    tf_normalized = {term: count / len(tokens) for term, count in tf_counts.items() for tokens in corpus}

    return tf_normalized

def calculate_idf(corpus):
    idf_counts = defaultdict(int)
    total_documents = len(corpus)

    for instance in tqdm(corpus):
        tokens = set(instance['input_ids'].squeeze().tolist())
        for token in tokens:
            idf_counts[token] += 1

    idf_normalized = {term: math.log(total_documents / count) for term, count in idf_counts.items()}

    return idf_normalized

def calculate_tf_idf(corpus):
    tf = calculate_tf(corpus)
    print('tf done')
    idf = calculate_idf(corpus)
    print('idf done')

    tf_idf = {term: tf[term] * idf[term] for term in tqdm(tf.keys())}

    return tf_idf



In [ ]:
tf_idf_scores = calculate_tf_idf(dataloader)
print(tf_idf_scores)


In [ ]:
print(tf_idf_scores)